Confirm than computations that are repeated on data groups (like in box plots) can be any faster if all of them are done at the sime time.

In [25]:
import os
import time

import create
import dask.array as da

fn = "t4e6_raw.hdf5"

if not os.path.exists(fn):
    create.t4e6_raw(fn)

ar = create.read_raw(fn)

# to go over the file once, just so that
# caching level is the same in all the code
print(ar.mean().compute())

0.9999539442321703


In [26]:

attr_col_n = 42
group_col_n = -1

group_col = ar[:, group_col_n]
attr_col = ar[:, attr_col_n]

t = time.time()
groups = da.unique(group_col).compute()
print("time to obtain groups", time.time() - t)

time to obtain groups 0.22876334190368652


In [27]:
print()
print("COMPUTE per group")

means = []
for i in groups:
    print("Group", i)
    t = time.time()
    m1 = attr_col[group_col == i].mean().compute()
    print("group time", time.time()-t)
    means.append(m1)
print(means)


COMPUTE per group
Group 0.0
group time 0.41014671325683594
Group 1.0
group time 0.438091516494751
Group 2.0
group time 0.45922207832336426
Group 3.0
group time 0.4527897834777832
[0.49499428248762584, 0.5040881892648345, 0.502314728293208, 0.5062434784045853]


In [28]:
print()
print("COMPUTE all results")

t = time.time()
means = []
for i in groups:
    print("Group", i)
    m1 = attr_col[group_col == i].mean()
    means.append(m1)
r = da.stack(means)
print(r.compute())
print("combined time", time.time()-t)


COMPUTE all results
Group 0.0
Group 1.0
Group 2.0
Group 3.0
[0.49499428 0.50408819 0.50231473 0.50624348]
combined time 0.42580199241638184
